### Install dependencies

In [1]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import app as app_file
import setup as setup_file
import dash_core_components as dcc
import pickle
from Configuration import *

### Serve Jupyter Dash

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Define your static parameters

In [11]:
configs = Config()

In [12]:
setup_applet = setup_file.create_app(configs)
setup_applet.run_server(mode='inline', port=8893)

### View your sample population

In [13]:
sample_pop = configs.enc.initialize()
print(sample_pop)

[{'gene': array([1, 6, 7, 2, 3, 5, 8, 4, 9, 0]), 'fitness': array([0])}
 {'gene': array([0, 5, 7, 4, 2, 6, 8, 9, 3, 1]), 'fitness': array([0])}
 {'gene': array([4, 0, 2, 3, 1, 8, 5, 9, 7, 6]), 'fitness': array([0])}
 {'gene': array([6, 9, 4, 1, 3, 0, 2, 5, 7, 8]), 'fitness': array([0])}
 {'gene': array([1, 2, 8, 3, 0, 5, 7, 6, 9, 4]), 'fitness': array([0])}]


### Define your evaluation function

Below is an example fitness evaluation function for the single-objective Traveling Salesman Problem (TSP)

```
def TSP(ind):
    dist = np.loadtxt('dist.txt')

    ind['fitness'][0] += dist[0][ind['gene'][0]]
    ind['fitness'][0] += np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum()
    ind['fitness'][0] += dist[ind['gene'][-1]][0]

    return ind
```

In [14]:
def TSP(pop):
    dist = np.loadtxt('dist.txt')

    for ind in pop:
        ind['fitness'][0] += dist[0][ind['gene'][0]]
        ind['fitness'][0] += np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum()
        ind['fitness'][0] += dist[ind['gene'][-1]][0]

    return pop

sample_pop = TSP(sample_pop)
print(sample_pop[0])

{'gene': array([1, 6, 7, 2, 3, 5, 8, 4, 9, 0]), 'fitness': array([4242])}


### Define a custom phenotypic representation

In [15]:
def network(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][0]] + [locs[0][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[0][0]],
                                     y=[locs[1][0]] + [locs[1][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[1][0]],
                                     mode='lines')])
    return fig

print(network(sample_pop[0]))

Figure({
    'data': [{'mode': 'lines',
              'type': 'scatter',
              'x': [48.0, 28.0, 84.0, 97.0, 55.0, 58.0, 72.0, 59.0, 15.0, 3.0,
                    48.0, 1.0],
              'y': [84.0, 36.0, 8.0, 44.0, 30.0, 80.0, 29.0, 89.0, 88.0, 87.0,
                    84.0, 16.0]}],
    'layout': {'template': '...'}
})


### Save your static configurations

If you are satisfied with your evaluation function and custom visualization, save it to your configurations

In [16]:
configs.eval, configs.vis = TSP, network

### Run the application

In [17]:
main_app = app_file.create_app(configs)
main_app.run_server(port=8893)

Dash app running on http://127.0.0.1:8893/


In [11]:
configs.sel

<bound method Single.rank_based of <Configuration.Single object at 0x7fc9532cbc10>>